In [24]:
# analyse label cnt
import json
import pandas as pd

d = json.loads(open('../data/train_data.json').read())
df_data = []
for k in d:
    df_data.append([k, len(d[k])])
print(df_data[:5])

[['w_6cb1882', 2], ['w_cbdae16', 2], ['w_dcc14b3', 2], ['w_185044d', 2], ['w_5ccee38', 1]]


In [25]:
df = pd.DataFrame(df_data, columns=['l', 'cnt'])
df.head()

,l,cnt
0,w_6cb1882,2
1,w_cbdae16,2
2,w_dcc14b3,2
3,w_185044d,2
4,w_5ccee38,1


In [26]:
top5_labels = df.sort_values('cnt').l.values[-5:]
print(top5_labels)

['w_0369a5c' 'w_9c506f6' 'w_9b5109b' 'w_23a388d' 'new_whale']


In [27]:
df.describe()

,cnt
count,5005.000000
mean,5.067133
std,136.634756
min,1.000000
25%,1.000000
50%,2.000000
75%,3.000000
max,9664.000000


In [28]:
for i in [2, 3, 4]:
    print(i, len(df[df.cnt < i]))

2 2073
3 3358
4 3926


In [29]:
# analyse public sub
pub_df = pd.read_csv('../data/pub_855.csv')
pub_df.head(10)

,Image,Id
0,00028a005.jpg,w_dec7ffd new_whale w_ab44ae4 w_2a0e817 w_9119b08
1,000dcf7d8.jpg,w_4132bb8 new_whale w_3de579a w_9119b08 w_d7aef56
2,000e7c7df.jpg,w_94c3bf0 new_whale w_3de579a w_d7aef56 w_17b0d3a
3,0019c34f4.jpg,w_a335fc2 w_c99807e new_whale w_59052ad w_9119b08
4,001a4d292.jpg,new_whale w_5b40032 w_82e7319 w_20eeb29 w_ce66f9a
5,00247bc36.jpg,new_whale w_d7aef56 w_2284d86 w_17b0d3a w_9119b08
6,0027089a4.jpg,w_7e2eb3d w_2284d86 new_whale w_a9304b9 w_3de579a
7,002de4d94.jpg,new_whale w_9119b08 w_a9304b9 w_3879fa0 w_17b0d3a
8,002f52f0c.jpg,new_whale w_53feb91 w_3879fa0 w_d7aef56 w_17b0d3a
9,002fd89d4.jpg,new_whale w_261878d w_d6907f5 w_e8b5980 w_d0e6e7d


In [30]:
preds = pub_df.Id.values
preds = [p.split(' ') for p in preds]
print(preds[0])

for i in range(5):
    print('top', i)
    tmp_set = set()
    for p in preds:
        for element in p[:i+1]:
            tmp_set.add(element)
    print(len(tmp_set))
    print('----------')

['w_dec7ffd', 'new_whale', 'w_ab44ae4', 'w_2a0e817', 'w_9119b08']
top 0
3305
----------
top 1
3975
----------
top 2
4305
----------
top 3
4448
----------
top 4
4552
----------


In [31]:
# simple test, replace last 2 with top res in top5
def update_p(p_list, top5):
    tmp_p = p_list[:3]
    tmp_filter_top = [l for l in top5 if l not in tmp_p]
    tmp_p = tmp_p + tmp_filter_top[-2:]
    return tmp_p


# test
print(update_p(preds[0], top5_labels))

test_new_pred = [' '.join(update_p(p, top5_labels)) for p in preds]
new_df = pub_df.copy()
new_df.Id = test_new_pred
print(new_df.head(10))
new_df.to_csv('../submissions/replace_2.csv',index=False)

['w_dec7ffd', 'new_whale', 'w_ab44ae4', 'w_9b5109b', 'w_23a388d']
           Image                                                 Id
0  00028a005.jpg  w_dec7ffd new_whale w_ab44ae4 w_9b5109b w_23a388d
1  000dcf7d8.jpg  w_4132bb8 new_whale w_3de579a w_9b5109b w_23a388d
2  000e7c7df.jpg  w_94c3bf0 new_whale w_3de579a w_9b5109b w_23a388d
3  0019c34f4.jpg  w_a335fc2 w_c99807e new_whale w_9b5109b w_23a388d
4  001a4d292.jpg  new_whale w_5b40032 w_82e7319 w_9b5109b w_23a388d
5  00247bc36.jpg  new_whale w_d7aef56 w_2284d86 w_9b5109b w_23a388d
6  0027089a4.jpg  w_7e2eb3d w_2284d86 new_whale w_9b5109b w_23a388d
7  002de4d94.jpg  new_whale w_9119b08 w_a9304b9 w_9b5109b w_23a388d
8  002f52f0c.jpg  new_whale w_53feb91 w_3879fa0 w_9b5109b w_23a388d
9  002fd89d4.jpg  new_whale w_261878d w_d6907f5 w_9b5109b w_23a388d
